# Setting up

In [1]:
import torch
import json
import spacy
import numpy as np
from pprint import pprint
from spacy.language import Language
from spacy.tokens import Doc
from spacy.tokenizer import Tokenizer
import re
import unicodedata
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
device="cuda" if torch.cuda.is_available() else "cpu"
import torch.nn.functional as F
print(torch.__version__)

2.6.0+cu124


In [2]:
# Setting up text preprocessing functions
def unicodeToAscii(s):
    # Convert a Unicode string 's' to plain ASCII.
    # This is done by first normalizing the string into its decomposed form using 'NFD',
    # which separates characters from their accents. Then, it filters out all nonspacing marks (Mn).
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalize_whitespace(text):
  return re.sub(r'\s+', ' ', text).strip()

def preprocess_sentence(s:str) -> str:
    """
    Preprocesses sentence text for consistency
    """
    s = s.strip()
    s = normalize_whitespace(s)
    s = unicodeToAscii(s)
    s = s.strip()
    return s

## Preparing Dataset

### Preparing Tokenizer

We are using spacy pre-built NLP pipeline 'en_core_web_lg' with a few changes:

*   Using  whitespace tokenisation
*   Making use of the Named Entity Recognition (NER) component to identify tokens which belong to Geopolitical Entity (GPE) and Organisation (ORG) Entity Type, to make sure that words belonging to these entity types remain a single token for easier implementation of the tagging problem

For the Linear model, we are utilising the static embeddings provided by spacy which uses floret word vectors by default. For the Feedforward, LSTM and Transformer models, we are utilising the contextual embeddings provided by spacy which use CNN network to generate them. We are not fine-tuning the embeddings.


In [4]:
!python -m spacy download en_core_web_lg

nlp = spacy.load('en_core_web_lg')


# Create a custom component to merge entities
@Language.component("entity_merger")
def entity_merger(doc):
    """
    Custom component of the spacy nlp pipeline which merges geopolitical and organisation entity tokens into a single token
    For example: 'New York' would normally be split into 2 tokens 'New' and 'York' but this will combine into a single 'New York' token
    This is implemented because variables could have the value like 'New York' and for effective tagging we aim to keep the tokenisation scheme consistent to the dataset
    """
    # Iterate over the entities in reverse order (to avoid index issues when merging)
    with doc.retokenize() as retokenizer:
        for ent in reversed(list(doc.ents)):
            # Merge the entity tokens into one token
            if(ent.label_ in ["GPE", "ORG"]):
                attrs = {"LEMMA": ent.text}
                retokenizer.merge(ent, attrs=attrs)
    return doc

# Add the custom component after NER
nlp.add_pipe("entity_merger", after="ner")

def whitespace_tokenizer(nlp):
  """
  Create a custom tokenizer that splits only on whitespace
  """
  return Tokenizer(nlp.vocab, token_match=re.compile(r'\S+').match)

nlp.tokenizer = whitespace_tokenizer(nlp)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 4.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


### Preparing Pytorch Datasets and DataLoaders

We are using a custom pytorch dataset to standardise the dataloading process. The dataset reads the json file and determines which examples to keep depending on the split type i.e. 'question' or 'query' and split i.e. 'train', 'dev', or 'test'. The sentences and sql queries with variables, the variable names and their corresponding values are stored for each sample. The variables in sentences and sql queries are replaced by their values and also stored. The shortest sql query by length is also identified for training. Depending on the embed type i.e. 'static' or 'context', static token vectors or contextual token embeddings are stored. Disinct sql query templates and variables names are also identified and converted to ids for classification and tagging. Dataloader is used for batching which uses padding ot ensure all input tokens and tagging labels are the same size

In [5]:
class ATISClassificationDataset(Dataset):

    def __init__(self, dataset_loc, nlp, split_type='query', split=['train'], tokenizer=None, embed_type='static'):

        self.nlp = nlp

        if(embed_type not in ['static', 'context']):
          raise ValueError(f"{embed_type=} is not valid. Please choose from 'static' or 'context'")

        self.embed_type = embed_type
        self.data = []

        self.variable_names = set()
        self.sql_templates = set()

        # Reads Json
        with open(dataset_loc) as f:
            dataset_json = json.load(f)

        for sample in dataset_json:

            processed_sample = {}

            # All valid sql queries for this examples soretd by length
            sql = sorted(sample['sql'],key=len)

            # Adds shortest sql query template to list of sql templates
            self.sql_templates.add(sql[0])

            # Check query split
            query_split = sample['query-split']
            if split_type == "query" and query_split not in split:
                continue

            # Adds to variables set
            variables_metadata = sample["variables"]
            for var in variables_metadata:
                self.variable_names.add(var.get("name"))

            # Process each sentence
            for sentence in sample['sentences']:

                # Check question split
                if split_type == "question" and sentence['question-split'] not in split:
                    continue

                # variables/placeholder mapping dictionary
                variables = sentence['variables']

                # Sentence text with variables/placeholders
                text_with_vars = sentence['text']

                # Replacing variables/placeholders in current sentence and sql query with their values from the variables dictionary
                text_with_vars_replaced = text_with_vars
                sql_with_vars_replaced = sql.copy()

                # Replace sentence and all sql variables with their values
                for var in variables:
                    text_with_vars_replaced = text_with_vars_replaced.replace(var,variables[var])

                    # sql_with_vars_replaced = sql_with_vars_replaced.replace(var,variables[var])
                    sql_with_vars_replaced = [query.replace(var,variables[var]) for query in sql_with_vars_replaced]

                # Tagging expected output
                sentence_var_tagging_labels = []
                for word in text_with_vars.split():
                    if(word in variables):
                        # Use variable name as tag
                        sentence_var_tagging_labels.append(word)
                    else:
                        # Use 'no_var' for non-variable tokens
                        sentence_var_tagging_labels.append("no_var")

                # Appends preprocessed dictionary of current sentence to the processesed_dataset list
                self.data.append({
                    "text_with_vars": text_with_vars,
                    "text_with_vars_replaced":text_with_vars_replaced,
                    "tagging_labels":sentence_var_tagging_labels,
                    "variables":variables,
                    "sql_with_vars": sql,
                    "shortest_sql_with_vars":sql[0],
                    "sql_with_vars_replaced": sql_with_vars_replaced,
                    "shortest_sql_with_vars_replaced": sql_with_vars_replaced[0]
                })

        # Setup tagging label encoder
        # For tagging task - include all variable names plus "no_var" for non-variables
        all_tags = ["no_var"] + list(self.variable_names)
        all_tags = np.array(all_tags)
        self.tag_encoder = LabelEncoder()
        self.tag_encoder.fit(all_tags)

        # Setup SQL template label encoder
        self.sql_encoder = LabelEncoder()
        self.sql_encoder.fit(np.array(list(self.sql_templates)))

        # Process sentences using spacy nlp pipeline to get docs, corresponding label ids and tag ids
        self.docs = []
        self.tag_labels = []
        self.sql_labels = []

        for sample in self.data:
            # Convert sentence with variables into spacy Doc
            doc = self.nlp(preprocess_sentence(sample['text_with_vars_replaced']))
            self.docs.append(doc)

            # Covert variable tags into ids for tagging
            tags = sample["tagging_labels"]
            self.tag_labels.append(self.tag_encoder.transform(tags))

            # Convert sql query with variables into ids for classification
            sql_template = sample["shortest_sql_with_vars"]
            self.sql_labels.append(self.sql_encoder.transform([sql_template])[0])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        """
        Get a single sample from the dataset
        """

        doc = self.docs[idx]
        raw_item = self.data[idx]

        # Create tensor from tokens
        if(self.embed_type == 'context'):
          # Get contextual embeddings from spacy Doc
          token_vectors = torch.tensor(doc.tensor, dtype=torch.float32)

        elif(self.embed_type == 'static'):
          # Get static token vectors from spacy Doc
          token_vectors = torch.tensor([token.vector for token in doc], dtype=torch.float32)

        # Get token texts (needed for variable replacement during inference)
        tokens = [token.text for token in doc]

        # Get tag labels
        tag_labels = torch.tensor(self.tag_labels[idx], dtype=torch.long)

        # Get SQL label
        sql_label = torch.tensor(self.sql_labels[idx], dtype=torch.long)

        return {
            "token_vectors": token_vectors,
            "tokens": tokens,
            "tag_labels": tag_labels,
            "sql_label": sql_label,
            "raw_item": raw_item,
            "doc_len": len(doc),
            "true_sql_text": raw_item['shortest_sql_with_vars_replaced'] # For inference
        }

    def get_dataloader(self, batch_size=32, shuffle=True, num_workers=0):
        """Helper function to create a DataLoader with custom collate function"""
        return DataLoader(
            self,
            batch_size=batch_size,
            shuffle=shuffle,
            num_workers=num_workers,
            collate_fn=self.collate_fn,
            pin_memory=True
        )

    def collate_fn(self, batch):
      """Custom collate function that pads sequences to the longest in the batch"""

      # Find max length for both token vectors and tag labels in this batch
      max_token_len = max([item["doc_len"] for item in batch])
      max_tag_len = max([len(item["tag_labels"]) for item in batch])

      # Ensure both use the same max length for consistent padding
      max_len = max(max_token_len, max_tag_len)

      # Prepare lists to collect tensors and data
      token_vectors_list = []
      tag_labels_list = []
      attention_masks = []
      sql_labels = []
      tokens_list = []
      raw_items = []
      true_sql_text_list = []

      for item in batch:
          # Get original tensors and data
          token_vecs = item["token_vectors"]
          tags = item["tag_labels"]
          tokens = item["tokens"]

          # Create attention mask (1 for real tokens, 0 for padding)
          seq_len = len(token_vecs)
          tag_len = len(tags)
          attention_mask = torch.ones(seq_len, dtype=torch.long)

          # If padding is needed for tokens
          if(seq_len < max_len):
            # Pad token vectors
            padding = torch.zeros(max_len - seq_len, token_vecs.shape[1], dtype=torch.long)
            token_vecs = torch.cat([token_vecs,padding], dim=0)
            attention_mask = torch.cat([attention_mask, torch.zeros(max_len - seq_len, dtype=torch.long)])

            # Pad tokens list
            tokens.extend([""] * (max_len - seq_len))

          # Pad tag labels separately to ensure they all have the same length
          if(tag_len < max_len):
            padding = torch.zeros(max_len - tag_len, dtype=torch.long)
            tags = torch.cat([tags, padding], dim=0)

          # Add to lists
          token_vectors_list.append(token_vecs)
          tag_labels_list.append(tags)
          attention_masks.append(attention_mask)
          sql_labels.append(item["sql_label"])
          tokens_list.append(tokens)
          raw_items.append(item["raw_item"])
          true_sql_text_list.append(item["true_sql_text"])

      # Stack tensors
      return {
          "token_vectors": torch.stack(token_vectors_list),
          "tag_labels": torch.stack(tag_labels_list),
          "attention_mask": torch.stack(attention_masks),
          "sql_labels": torch.stack(sql_labels),
          "tokens": tokens_list,
          "raw_items": raw_items,
          "true_sql_texts":true_sql_text_list
      }

    def get_tag_vocab_size(self):
        """Returns the size of the tag vocabulary"""
        return len(self.tag_encoder.classes_)

    def get_sql_vocab_size(self):
        """Returns the number of unique SQL templates"""
        return len(self.sql_encoder.classes_)

    def get_vector_dim(self):
        """Returns the dimensionality of token vectors"""
        if(self.embed_type == 'context'):
          return self.nlp.get_pipe("tok2vec").model.get_dim("nO")
        elif(self.embed_type == 'static'):
          return self.nlp.vocab.vectors_length

    def decode_tag(self, tag_id):
      """Convert tag ID back to original variable name or 'no_var' """
      try:
          return self.tag_encoder.inverse_transform([tag_id])[0]
      except ValueError:
          # Handle unseen labels
          return "unknown"

    def decode_sql_template(self, sql_id):
        """Convert SQL template ID back to original SQL template"""
        try:
          return self.sql_encoder.inverse_transform([sql_id])[0]
        except ValueError:
          return "Unknown sql template"

# Model Setup

## Linear Model

The Linear Model has a separate model for variable tagging and SQL template classification. The Linear model is a Multi-Layer Perceptron with no non-linear activation to keep the model linear.

In [6]:
import torch
import torch.nn as nn

class LinearTagger(nn.Module):
    def __init__(self, input_dim: int, tag_vocab_size: int,hidden_sizes=None, dropout: float = 0):
        """
        Linear model for token-level tagging.

        Args:
            input_dim: Dimensionality of input token vectors
            tag_vocab_size: Number of unique tags
            dropout: Dropout probability
        """
        super(LinearTagger, self).__init__()
        layers = []

        if not hidden_sizes:
            hidden_dim1 = 512
            hidden_dim2 = 256
            layers.append(nn.Linear(input_dim, hidden_dim1))
            layers.append(nn.Dropout(dropout))
            layers.append(nn.Linear(hidden_dim1, hidden_dim2))

            layers.append(nn.Dropout(dropout))

            layers.append(nn.Linear(hidden_dim2, tag_vocab_size))
        else:

            current_dim = input_dim
            for h_size in hidden_sizes:
                layers.append(nn.Linear(current_dim, h_size))
                layers.append(nn.Dropout(dropout))
                current_dim = h_size
            layers.append(nn.Linear(current_dim, tag_vocab_size))

        self.layers = nn.Sequential(*layers)


    def forward(self, token_vectors):
        """
        Args:
            token_vectors: [batch_size, seq_len, input_dim]

        Returns:
            tag_logits: [batch_size, seq_len, tag_vocab_size]
        """

        tag_logits = self.layers(token_vectors)
        return tag_logits

class LinearClassifier(nn.Module):
    def __init__(self, input_dim: int, sql_vocab_size: int,hidden_sizes=None, dropout: float = 0):
        """
        Linear model for SQL template classification.

        Args:
            input_dim: Dimensionality of input token vectors
            sql_vocab_size: Number of SQL templates
            dropout: Dropout probability
        """
        super(LinearClassifier, self).__init__()
        layers = []

        if not hidden_sizes:
            hidden_dim1 = 512
            hidden_dim2 = 256
            layers.append(nn.Linear(input_dim, hidden_dim1))
            layers.append(nn.Dropout(dropout))
            layers.append(nn.Linear(hidden_dim1, hidden_dim2))
            layers.append(nn.Dropout(dropout))
            layers.append(nn.Linear(hidden_dim2, sql_vocab_size))

        else:
            current_dim = input_dim
            for h_size in hidden_sizes:
                layers.append(nn.Linear(current_dim, h_size))
                layers.append(nn.Dropout(dropout))
                current_dim = h_size
            layers.append(nn.Linear(current_dim, sql_vocab_size))

        self.layers = nn.Sequential(*layers)

    def forward(self, token_vectors, attention_mask=None):
        """
        Args:
            token_vectors: [batch_size, seq_len, input_dim]
            attention_mask: [batch_size, seq_len] (1 = real token, 0 = padding)

        Returns:
            sql_logits: [batch_size, sql_vocab_size]
        """
        if attention_mask is not None:
            # Apply mask by setting padding tokens to a very small number before max pooling
            masked_vectors = token_vectors.masked_fill(~attention_mask.unsqueeze(-1).bool(), -float('inf'))
            pooled = masked_vectors.max(dim=1)[0] # Get values, ignore indices
        else:
            pooled = token_vectors.max(dim=1)[0]

        sql_logits = self.layers(pooled)
        return sql_logits



## Feedforward Model

The feed-forward model has a separate model for variable tagging and SQL template classification. Both models are Multi-Layer Perceptrons with GELU activation. Dropout and Layer normalisation is used for better generalisation.

In [7]:
import torch
import torch.nn as nn

class FFTagger(nn.Module):
    def __init__(self, input_dim: int, tag_vocab_size: int,hidden_sizes=None, dropout: float = 0.1, use_layernorm: bool = True):
        """
        Feedforward model for token-level tagging.

        Args:
            input_dim: Dimensionality of input token vectors
            tag_vocab_size: Number of unique tags
            dropout: Dropout probability
            use_layernorm: If true, uses Layer Normalisation after each hidden layer
        """
        super(FFTagger, self).__init__()
        layers = []

        if not hidden_sizes:
            hidden_dim1 = 512
            hidden_dim2 = 256
            layers.append(nn.Linear(input_dim, hidden_dim1))
            layers.append(nn.GELU())
            if use_layernorm:
                    layers.append(nn.LayerNorm(hidden_dim1)) # Normalize features of this layer
            layers.append(nn.Dropout(dropout))
            layers.append(nn.Linear(hidden_dim1, hidden_dim2))
            layers.append(nn.GELU())
            if use_layernorm:
                    layers.append(nn.LayerNorm(hidden_dim2)) # Normalize features of this layer
            layers.append(nn.Dropout(dropout))

            layers.append(nn.Linear(hidden_dim2, tag_vocab_size))
        else:

            current_dim = input_dim
            for h_size in hidden_sizes:
                layers.append(nn.Linear(current_dim, h_size))
                layers.append(nn.GELU())
                if use_layernorm:
                    layers.append(nn.LayerNorm(h_size)) # Normalize features of this layer
                layers.append(nn.Dropout(dropout))
                current_dim = h_size
            layers.append(nn.Linear(current_dim, tag_vocab_size))

        self.layers = nn.Sequential(*layers)


    def forward(self, token_vectors):
        """
        Args:
            token_vectors: [batch_size, seq_len, input_dim]

        Returns:
            tag_logits: [batch_size, seq_len, tag_vocab_size]
        """

        tag_logits = self.layers(token_vectors)
        return tag_logits

class FFClassifier(nn.Module):
    def __init__(self, input_dim: int, sql_vocab_size: int,hidden_sizes=None, dropout: float = 0.25, use_layernorm: bool = True):
        """
        Feedforward model for SQL template classification.

        Args:
            input_dim: Dimensionality of input token vectors
            sql_vocab_size: Number of SQL templates
            dropout: Dropout probability
            use_layernorm: If true, uses Layer Normalisation after each hidden layer
        """
        super(FFClassifier, self).__init__()
        layers = []

        if not hidden_sizes:
            hidden_dim1 = 512
            hidden_dim2 = 256
            layers.append(nn.Linear(input_dim, hidden_dim1))
            layers.append(nn.GELU())
            if use_layernorm:
                    layers.append(nn.LayerNorm(hidden_dim1)) # Normalize features of this layer
            layers.append(nn.Dropout(dropout))
            layers.append(nn.Linear(hidden_dim1, hidden_dim2))
            layers.append(nn.GELU())
            if use_layernorm:
                    layers.append(nn.LayerNorm(hidden_dim2)) # Normalize features of this layer
            layers.append(nn.Dropout(dropout))
            layers.append(nn.Linear(hidden_dim2, sql_vocab_size))
        else:
            current_dim = input_dim
            for h_size in hidden_sizes:
                layers.append(nn.Linear(current_dim, h_size))
                layers.append(nn.GELU())
                if use_layernorm:
                    layers.append(nn.LayerNorm(h_size)) # Normalize features of this layer
                layers.append(nn.Dropout(dropout))
                current_dim = h_size
            layers.append(nn.Linear(current_dim, sql_vocab_size))

        self.layers = nn.Sequential(*layers)

    def forward(self, token_vectors, attention_mask=None):
        """
        Args:
            token_vectors: [batch_size, seq_len, input_dim]
            attention_mask: [batch_size, seq_len] (1 = real token, 0 = padding)

        Returns:
            sql_logits: [batch_size, sql_vocab_size]
        """
        if attention_mask is not None:
            # Apply mask by setting padding tokens to a very small number before max pooling
            masked_vectors = token_vectors.masked_fill(~attention_mask.unsqueeze(-1).bool(), -float('inf'))
            pooled = masked_vectors.max(dim=1)[0]
        else:
            pooled = token_vectors.max(dim=1)[0]

        sql_logits = self.layers(pooled)
        return sql_logits



## LSTM Model

A multi-layered, bidirectional LSTM is used for tagging and classification. Output (concatenated from both directions) from each cell of the last LSTM layer is passed to a linear layer for tagging. Hidden state of the last cell of the last LSTM layer in both directons is concatenated and passed to Linear Layer for SQL template classification.

In [8]:
class LSTMTaggerClassifer(nn.Module):
    def __init__(self,
                 input_dim:int,
                 hidden_dim:int,
                 tag_vocab_size:int,
                 sql_vocab_size:int,
                 num_layers:int = 1,
                 dropout:float = 0.25):
        """
        LSTM model for both token tagging and SQL template classification

        Args:
            input_dim: Dimensionality of input vectors
            hidden_dim: Hidden dimension of LSTM
            tag_vocab_size: Size of tag vocabulary for tagging task
            sql_vocab_size: Number of unique SQL templates
            num_layers: Number of LSTM layers
            dropout: Dropout probability
        """
        super(LSTMTaggerClassifer, self).__init__()

        #LSTM layer
        self.lstm = nn.LSTM(
            input_dim,
            hidden_dim,
            num_layers=num_layers,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0,
            batch_first=True
        )

        # Dropout
        self.dropout = nn.Dropout(dropout)

        # Tagging layer (token classification)
        # hidden * 2 is for bidirectionality
        self.tag_classifier = nn.Linear(hidden_dim * 2, tag_vocab_size)

        # SQL Template classification layer
        # hidden * 2 is for bidirectionality
        self.sql_classifier = nn.Linear(hidden_dim * 2, sql_vocab_size)

    def forward(self, token_vectors, attention_mask=None):
        """
        Forward pass

        Args:
            token_vectors: Token vectors from SpaCy [batch_size, seq_len, input_dim]
            attention_mask: Attention mask indicating valid tokens [batch_size, seq_len]

        Returns:
            tag_logits: Token classification logits [batch_size, seq_len, tag_vocab_size]
            sql_logits: SQL template classification logits [batch_size, sql_vocab_size]
        """
        batch_size = token_vectors.shape[0]
        seq_len = token_vectors.shape[1]

        # Ignore padded vectrors in batch
        packed = nn.utils.rnn.pack_padded_sequence(token_vectors, attention_mask.sum(dim=1).cpu().long(), batch_first=True, enforce_sorted=False)
        # LSTM
        output, (hidden, _) = self.lstm(packed)
        # Unpack output back to padded sequences
        output, _ = nn.utils.rnn.pad_packed_sequence(output, batch_first=True,total_length=seq_len)
        # Appply dropout to output
        output = self.dropout(output)
        # Feedforward layer to get tagging logits
        tag_logits = self.tag_classifier(output)  # shape: (B, T, num_tags)
        # tag_logits = self.tag_classifier(F.gelu(output))  # shape: (B, T, num_tags)

        # Classification from last hidden states (concat of both directions)
        final_hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)
        # Apply dropout to final hidden output
        # final_hidden = self.dropout(final_hidden)
        # Feedforward layer to get SQL classification
        sql_logits = self.sql_classifier(final_hidden)
        # sql_logits = self.sql_classifier(F.gelu(final_hidden))

        return tag_logits, sql_logits

## Transfomer Model

The Transformer model is an encoder only model. It uses the outputs of each token of the encoder passed to a linear layer for tagging. For classification, it uses the output of a special [CLS] token passed to a linear layer, similar to BERT. The [CLS] token is appended to the input sentence

In [9]:
class TransformerTaggerClassifier(nn.Module):
  "Transformer Encoder for Tagging and Classification"

  def __init__(self, input_dim, hidden_dim, tag_vocab_size, sql_vocab_size, num_layers=2, num_heads=6, dropout=0.2):
    super().__init__()

    self.project = nn.Linear(input_dim, hidden_dim) if input_dim != hidden_dim else nn.Identity()

    encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads, dropout=dropout, batch_first=True,)
    self.encoder = nn.TransformerEncoder(encoder_layer, num_layers)

    self.tag_classifier = nn.Linear(hidden_dim, tag_vocab_size)
    self.sql_classifier = nn.Linear(hidden_dim, sql_vocab_size)

    # BERT style [CLS] token for classification
    self.cls_token = nn.Parameter(torch.randn(1, 1, hidden_dim))
    self.dropout = nn.Dropout(dropout)

  def forward(self, token_vectors, attention_mask=None):
    # x: [batch, seq_len, input_dim]
    x = self.dropout(self.project(token_vectors))
    batch_size = x.size(0)

    # Add [CLS] token
    cls = self.cls_token.expand(batch_size, -1, -1) # [batch, 1, hidden_dim]
    x = torch.cat([cls, x], dim=1)  # [batch, seq_len+1, hidden_dim]

    # Update attention mask
    if attention_mask is not None:
        cls_mask = torch.ones(batch_size, 1, device=attention_mask.device, dtype=attention_mask.dtype)
        attention_mask = torch.cat([cls_mask, attention_mask], dim=1)
        key_padding_mask = attention_mask == 0
    else:
        key_padding_mask = None

    x = self.encoder(x, src_key_padding_mask=key_padding_mask)

    # exclude CLS for token tagging
    tag_logits = self.tag_classifier(x[:, 1:])

    # use CLS for classification
    sql_logits = self.sql_classifier(x[:, 0])

    return tag_logits, sql_logits



In [218]:
import math

class PositionalEncoding(nn.Module):
    r"""Inject some information about the relative or absolute position of the tokens in the sequence.
        The positional encodings have the same dimension as the embeddings, so that the two can be summed.
        Here, we use sine and cosine functions of different frequencies.
    .. math:
        \text{PosEncoder}(pos, 2i) = sin(pos/10000^(2i/d_model))
        \text{PosEncoder}(pos, 2i+1) = cos(pos/10000^(2i/d_model))
        \text{where pos is the word position and i is the embed idx)
    Args:
        d_model: the embed dim (required).
        dropout: the dropout value (default=0.1).
        max_len: the max. length of the incoming sequence (default=5000).
    Examples:
        >>> pos_encoder = PositionalEncoding(d_model)
    """

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        r"""Inputs of forward function
        Args:
            x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        Examples:
            >>> output = pos_encoder(x)
        """

        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class TransformerTaggerClassifierStatic(nn.Module):
  "Transformer Encoder for Tagging and Classification using static embedding. Adding Poistional encoding"

  def __init__(self, input_dim, hidden_dim, tag_vocab_size, sql_vocab_size, num_layers=2, num_heads=6, dropout=0.2):
    super().__init__()

    self.project = nn.Linear(input_dim, hidden_dim) if input_dim != hidden_dim else nn.Identity()
    self.pos_encoder = PositionalEncoding(hidden_dim)
    encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads, dropout=dropout, batch_first=True,)
    self.encoder = nn.TransformerEncoder(encoder_layer, num_layers)

    self.tag_classifier = nn.Linear(hidden_dim, tag_vocab_size)
    self.sql_classifier = nn.Linear(hidden_dim, sql_vocab_size)

    # BERT style [CLS] token for classification
    self.cls_token = nn.Parameter(torch.randn(1, 1, hidden_dim))
    self.dropout = nn.Dropout(dropout)

  def forward(self, token_vectors, attention_mask=None):
    # x: [batch, seq_len, input_dim]
    x = self.dropout(self.project(token_vectors))
    # Apply positional encoding
    x = self.pos_encoder(x)
    batch_size = x.size(0)

    # Add [CLS] token
    cls = self.cls_token.expand(batch_size, -1, -1) # [batch, 1, hidden_dim]
    x = torch.cat([cls, x], dim=1)  # [batch, seq_len+1, hidden_dim]

    # Update attention mask
    if attention_mask is not None:
        cls_mask = torch.ones(batch_size, 1, device=attention_mask.device, dtype=attention_mask.dtype)
        attention_mask = torch.cat([cls_mask, attention_mask], dim=1)
        key_padding_mask = attention_mask == 0
    else:
        key_padding_mask = None

    x = self.encoder(x, src_key_padding_mask=key_padding_mask)

    # exclude CLS for token tagging
    tag_logits = self.tag_classifier(x[:, 1:])

    # use CLS for classification
    sql_logits = self.sql_classifier(x[:, 0])

    return tag_logits, sql_logits

# Training models

Training function for LSTM and Transformer models. CrossEntropy Loss from both tagging ang Classification is addded to get the loss which is to be minimised.

In [10]:
def train_model(model, train_loader, val_loader=None, epochs=1-0, lr=1e-3,weight_decay=0.01,gamma=0.1,steps=20, warmup_epochs=10, amsgrad=False, device="cuda" if torch.cuda.is_available() else "cpu"):
    """
    Train LSTM and Transformer model
    """
    model = model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay, amsgrad=amsgrad)
    # StepLR: Reduces learning rate every 10 epochs by a factor of 0.1
    # ExponentialLR reduces learning by gammma
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)
    tag_criterion = nn.CrossEntropyLoss(ignore_index=0)
    sql_criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        # Set model to training model
        model.train()
        total_loss = 0

        # Training loop
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
          # Get batch data
          token_vectors = batch["token_vectors"].to(device)
          attention_mask = batch["attention_mask"].to(device)
          tag_labels = batch["tag_labels"].to(device)
          sql_labels = batch["sql_labels"].to(device)

          # Forward pass
          tag_logits, sql_logits = model(token_vectors, attention_mask)

          # Handle dimension mismatches
          tag_seq_len = tag_logits.size(1)
          label_seq_len = tag_labels.size(1)

          if tag_seq_len != label_seq_len:
              # Either pad or truncate logits/labels to match
              if tag_seq_len < label_seq_len:
                  # If logits are shorter, truncate labels
                  tag_labels = tag_labels[:, :tag_seq_len]
              else:
                  # If labels are shorter, truncate logits
                  tag_logits = tag_logits[:, :label_seq_len, :]

          tag_loss = tag_criterion(tag_logits.view(-1, tag_logits.size(-1)), tag_labels.view(-1))
          sql_loss = sql_criterion(sql_logits, sql_labels)
          loss = tag_loss + sql_loss

          # Backward pass
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          total_loss += loss.item()
        # Warm start
        if epoch+1 > warmup_epochs:
          scheduler.step()

        avg_loss = total_loss/len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Training Loss: {avg_loss:.4f}, Learning Rate: {optimizer.param_groups[0]['lr']}")

    return model

Training function for Linear and Feedforward models. Both tagging and classification models are trained at the same time. The loss from tagging and classification is added to compute overall loss which is to minimised.

In [11]:
def train_dual_model(tagger_model, classifier_model, train_loader, val_loader=None,
                     epochs=10, lr=1e-3,weight_decay=0.01,gamma=0.1,steps=20, warmup_epochs=10,
                     device="cuda" if torch.cuda.is_available() else "cpu"):
    """
    Train separate tagger and classifier models.
    """
    tagger_model = tagger_model.to(device)
    classifier_model = classifier_model.to(device)

    optimizer = optim.AdamW(
        list(tagger_model.parameters()) + list(classifier_model.parameters()), lr=lr, weight_decay=weight_decay, amsgrad=False
    )
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)

    tag_criterion = nn.CrossEntropyLoss()
    sql_criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        tagger_model.train()
        classifier_model.train()
        total_loss = 0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            token_vectors = batch["token_vectors"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            tag_labels = batch["tag_labels"].to(device)
            sql_labels = batch["sql_labels"].to(device)

            # Forward pass through both models
            tag_logits = tagger_model(token_vectors)  # [B, T, tag_vocab]
            sql_logits = classifier_model(token_vectors, attention_mask)  # [B, sql_vocab]

            # Align tag_logits and tag_labels
            tag_seq_len = tag_logits.size(1)
            label_seq_len = tag_labels.size(1)
            if tag_seq_len != label_seq_len:
                if tag_seq_len < label_seq_len:
                    tag_labels = tag_labels[:, :tag_seq_len]
                else:
                    tag_logits = tag_logits[:, :label_seq_len, :]

            # Compute losses
            tag_loss = tag_criterion(tag_logits.view(-1, tag_logits.size(-1)), tag_labels.view(-1))
            sql_loss = sql_criterion(sql_logits, sql_labels)
            loss = tag_loss + sql_loss

            # Calculating gradients and updating weights
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        if epoch+1 > warmup_epochs:
          scheduler.step()
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Training Loss: {avg_loss:.4f}, Learning Rate: {optimizer.param_groups[0]['lr']}")

    return tagger_model, classifier_model


# Evaluating Models

Class for evaluation and inference. Can used to genrate SQL for a single input sentence or a batch. When batching is used using the DatLoader, accuracy is also checked with comparing combined putput of the tagging ang classification with the list of valid SQL queries for that input sentence.



In [53]:
class SQLPipeline:
    def __init__(self,model, dataset,embed_type="static", device="cuda" if torch.cuda.is_available() else "cpu"):
        """
        End-to-end evaluation and inference pipeline. Handles single model for tagging and classification and dual models.
        Can be used for onference and evaluation
        """
        if(isinstance(model,nn.Module)):
          self.model = model.to(device)
          self.model.eval()
        if(isinstance(model,list)):
          self.tagger = model[0].to(device)
          self.tagger.eval()
          self.classifier = model[1].to(device)
          self.classifier.eval()

        self.dataset = dataset
        self.device = device
        self.embed_type = embed_type

    def predict(self, text, nlp=None):
        """
        Generate SQL query for input text

        Args:
            text: Input text query
            nlp: Optional SpaCy pipeline (if not provided, uses the dataset's)

        Returns:
            Dict containing the predicted SQL query, identified variables, and template
        """

        nlp = nlp or self.dataset.nlp

        # Process text with SpaCy
        doc = nlp(text)

        # Extract token vectors and texts
        # squeeze(0) used add batch dimension bcos model expects batch but we are processing a single sample
        if(self.embed_type == "context"):
          token_vectors = torch.tensor(doc.tensor, dtype=torch.float32).unsqueeze(0).to(self.device)
        elif(self.embed_type == "static"):
          token_vectors = torch.tensor([token.vector for token in doc], dtype=torch.float32).unsqueeze(0).to(self.device)
        elif(self.embed_type == "none"):
          raise NotImplementedError(f"Using token ids not implemented")
        else:
          raise ValueError(f"{self.embed_type} is not valid. Please choose from 'context', 'static', or 'none'")

        tokens = [token.text for token in doc]

        # Create attention mask
        attention_mask = torch.ones(1, len(doc), dtype=torch.float32).to(self.device)

        # Forward pass
        with torch.no_grad():
            if(hasattr(self,"model")):
              tag_logits, sql_logits =self.model(token_vectors, attention_mask)
            elif(hasattr(self,"tagger") and hasattr(self,"classifier")):
              tag_logits = self.tagger(token_vectors)
              sql_logits = self.classifier(token_vectors, attention_mask)

            # Get predicted tag for each token
            tag_preds = torch.argmax(tag_logits, dim=-1)[0].cpu().numpy()

            # Get predicted SQL template
            sql_pred = torch.argmax(sql_logits, dim=-1)[0].item()

        # Convert predictions back to human-readable form
        predicted_tags = [self.dataset.decode_tag(tag) for tag in tag_preds[:len(tokens)]]
        predicted_template = self.dataset.decode_sql_template(sql_pred)

        # Extract identified variables
        identified_variables = {}
        for i, (token, tag) in enumerate(zip(tokens, predicted_tags)):
            if tag not in ['no_var','unknown'] :
                # Tag is a variable name
                identified_variables[tag] = token

        # Replace variables in the SQL template
        final_sql = predicted_template
        for var_name, var_value in identified_variables.items():
            final_sql = final_sql.replace(var_name, var_value)

        return {
            "tokens": tokens,
            "predicted_tags": predicted_tags,
            "identified_variables": identified_variables,
            "predicted_template": predicted_template,
            "final_sql": final_sql
        }

    def batch_predict(self, dataloader):
        """
        Generate SQL queries for a batch of inputs and get accuracy

        Args:
            dataloader: DataLoader with test data

        Returns:
            List(dict): List of prediction results
            (float): Batch Accuracy
        """
        results = []
        total_samples = len(self.dataset)
        correct = 0
        for batch in tqdm(dataloader, desc="Generating SQL queries"):

            # Move batch to device
            token_vectors = batch["token_vectors"].to(self.device)
            attention_mask = batch["attention_mask"].to(self.device)
            tokens = batch["tokens"]
            raw_items = batch["raw_items"]

            # Forward pass
            with torch.no_grad():
                if(hasattr(self,"model")):
                  tag_logits, sql_logits =self.model(token_vectors, attention_mask)
                elif(hasattr(self,"tagger") and hasattr(self,"classifier")):
                  tag_logits = self.tagger(token_vectors)
                  sql_logits = self.classifier(token_vectors, attention_mask)

                # Get predictions
                tag_preds = torch.argmax(tag_logits, dim=-1).cpu().numpy()
                sql_preds = torch.argmax(sql_logits, dim=-1).cpu().numpy()

            # Process each item in batch
            for i in range(len(tokens)):
              # Get token and tag sequences
              item_tokens = tokens[i]
              # Filter out padding
              valid_tokens = [t for t in item_tokens if t]

              # Get predicted tags for valid tokens
              item_tags = [self.dataset.decode_tag(tag) for tag in tag_preds[i, :len(valid_tokens)]]

              # Get predicted SQL template
              predicted_template = self.dataset.decode_sql_template(sql_preds[i])

              # Extract identified variables
              identified_variables = {}
              for j, (token, tag) in enumerate(zip(valid_tokens, item_tags)):
                  if tag not in ['no_var','unknown']:
                      identified_variables[tag] = token

              # Replace variables in the SQL template
              final_sql = predicted_template
              for var_name, var_value in identified_variables.items():
                  final_sql = final_sql.replace(var_name, var_value)

              # Check if produces SQL query is correct
              correct_example = False
              if(final_sql in raw_items[i]["sql_with_vars_replaced"]):
                correct_example = True
                correct += 1
                # print(f"\nCorrrect PRediction:\n{final_sql}\nFor {' '.join([token for token in valid_tokens])}")



              # Add to results
              results.append({
                  "tokens": valid_tokens,
                  "predicted_tags": item_tags,
                  "identified_variables": identified_variables,
                  "predicted_template": predicted_template,
                  "final_sql": final_sql,
                  "raw_item": raw_items[i],
                  "correct_sql":raw_items[i]["sql_with_vars_replaced"],
                  "is_correct":correct_example
              })
        # print(f"Num Correct: {correct}\nNum Samples:{total_samples}")
        accuracy = correct/total_samples
        return results,accuracy

# Training and evaluating all 4 models

## Creating datasets and dataloaders for both questions and query split

### Question split using static embeddings

In [13]:
# Create datasets
question_static_train_val_dataset = ATISClassificationDataset(
    dataset_loc="atis.json",
    nlp=nlp,
    split_type="question",
    embed_type='static',
    split=["train","dev"]
)

question_static_test_dataset = ATISClassificationDataset(
    dataset_loc="atis.json",
    nlp=nlp,
    split_type="question",
    embed_type='static',
    split=["test"]
)

# Create dataloaders
question_static_train_val_loader = question_static_train_val_dataset.get_dataloader(batch_size=32)
question_static_test_loader = question_static_test_dataset.get_dataloader(batch_size=32)

### Question split using contextual embeddings

In [14]:
# Create datasets
question_context_train_val_dataset = ATISClassificationDataset(
    dataset_loc="atis.json",
    nlp=nlp,
    split_type="question",
    embed_type='context',
    split=["train","dev"]
)

question_context_test_dataset = ATISClassificationDataset(
    dataset_loc="atis.json",
    nlp=nlp,
    split_type="question",
    embed_type='context',
    split=["test"]
)

# Create dataloaders
question_context_train_val_loader = question_context_train_val_dataset.get_dataloader(batch_size=32)
question_context_test_loader = question_context_test_dataset.get_dataloader(batch_size=32)

### Query split using static embeddings

In [15]:
# Creata datasets with query split
query_static_train_val_dataset = ATISClassificationDataset(
    dataset_loc="atis.json",
    nlp=nlp,
    split_type="query",
    embed_type='static',
    split=["train","dev"]
)
query_static_test_dataset = ATISClassificationDataset(
    dataset_loc="atis.json",
    nlp=nlp,
    split_type="query",
    embed_type='static',
    split=["test"]
)

# Create dataloaders
query_static_train_val_loader = query_static_train_val_dataset.get_dataloader(batch_size=32)
query_static_test_loader = query_static_test_dataset.get_dataloader(batch_size=32)

### Query split using contextual embeddings

In [16]:
query_context_train_val_dataset = ATISClassificationDataset(
    dataset_loc="atis.json",
    nlp=nlp,
    split_type="query",
    embed_type='context',
    split=["train","dev"]
)

query_context_test_dataset = ATISClassificationDataset(
    dataset_loc="atis.json",
    nlp=nlp,
    split_type="query",
    embed_type='context',
    split=["test"]
)

# Create dataloaders
query_context_train_val_loader = query_context_train_val_dataset.get_dataloader(batch_size=32)
query_context_test_loader = query_context_test_dataset.get_dataloader(batch_size=32)

## Training and Evaluating Tagging and Classification models

### Training and Evaluating Linear models using question and query split

In [17]:
# Question split
# Model init
question_linear_tag_model = LinearTagger(
  input_dim=question_static_train_val_dataset.get_vector_dim(),
  tag_vocab_size=question_static_train_val_dataset.get_tag_vocab_size(),
  hidden_sizes=[256,128],
  dropout=0
)

question_linear_classifier_model = LinearClassifier(
  input_dim=question_static_train_val_dataset.get_vector_dim(),
  sql_vocab_size=question_static_train_val_dataset.get_sql_vocab_size(),
  hidden_sizes=[256,128],
  dropout=0
)
# Training
question_trained_linear_models = train_dual_model(
    tagger_model=question_linear_tag_model,
    classifier_model=question_linear_classifier_model,
    train_loader=question_static_train_val_loader,
    epochs=50,
    warmup_epochs=15,
    gamma=0.90,
    weight_decay=0.06
)

Epoch 1/50:   0%|          | 0/152 [00:00<?, ?it/s]<ipython-input-5-c9b2896365a8>:133: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  token_vectors = torch.tensor([token.vector for token in doc], dtype=torch.float32)
Epoch 1/50: 100%|██████████| 152/152 [00:05<00:00, 28.02it/s]


Epoch 1/50, Training Loss: 5.7158, Learning Rate: 0.001


Epoch 2/50: 100%|██████████| 152/152 [00:03<00:00, 41.06it/s]


Epoch 2/50, Training Loss: 4.1558, Learning Rate: 0.001


Epoch 3/50: 100%|██████████| 152/152 [00:03<00:00, 41.06it/s]


...

Epoch 48/50, Training Loss: 0.2921, Learning Rate: 3.090315438263264e-05


Epoch 49/50: 100%|██████████| 152/152 [00:03<00:00, 40.35it/s]


Epoch 49/50, Training Loss: 0.3016, Learning Rate: 2.7812838944369376e-05


Epoch 50/50: 100%|██████████| 152/152 [00:03<00:00, 41.04it/s]

Epoch 50/50, Training Loss: 0.2941, Learning Rate: 2.503155504993244e-05


In [54]:
# Evaluating on the test set to get accuracy
question_linear_sql_pipe = SQLPipeline(dataset=question_static_test_dataset,model=list(question_trained_linear_models),embed_type="static")
question_linear_results, question_linear_acc = question_linear_sql_pipe.batch_predict(dataloader=question_static_test_loader)
print(f"\nAccuracy of Linear model on question split test set: {question_linear_acc:.2%}")

Generating SQL queries: 100%|██████████| 14/14 [00:01<00:00,  9.67it/s]


Accuracy of Linear model on question split test set: 11.19%


In [19]:
# Query split
# Model init
query_linear_tag_model = LinearTagger(
  input_dim=query_static_train_val_dataset.get_vector_dim(),
  tag_vocab_size=query_static_train_val_dataset.get_tag_vocab_size(),
  hidden_sizes=[256,128]
)

query_linear_classifier_model = LinearClassifier(
  input_dim=query_static_train_val_dataset.get_vector_dim(),
  sql_vocab_size=query_static_train_val_dataset.get_sql_vocab_size(),
  hidden_sizes=[256,128]
)
# Training
query_trained_linear_models = train_dual_model(
    tagger_model=query_linear_tag_model,
    classifier_model=query_linear_classifier_model,
    train_loader=query_static_train_val_loader,
    epochs=50,
    warmup_epochs=15,
    gamma=0.90,
    weight_decay=0.06
)

Epoch 1/50: 100%|██████████| 155/155 [00:04<00:00, 32.66it/s]


Epoch 1/50, Training Loss: 5.5867, Learning Rate: 0.001


Epoch 2/50: 100%|██████████| 155/155 [00:03<00:00, 42.20it/s]


Epoch 2/50, Training Loss: 3.9004, Learning Rate: 0.001


Epoch 3/50: 100%|██████████| 155/155 [00:03<00:00, 41.60it/s]


...

Epoch 48/50, Training Loss: 0.2810, Learning Rate: 3.090315438263264e-05


Epoch 49/50: 100%|██████████| 155/155 [00:04<00:00, 32.91it/s]


Epoch 49/50, Training Loss: 0.2774, Learning Rate: 2.7812838944369376e-05


Epoch 50/50: 100%|██████████| 155/155 [00:03<00:00, 41.27it/s]

Epoch 50/50, Training Loss: 0.2798, Learning Rate: 2.503155504993244e-05


In [20]:
# Evaluating on the test set to get accuracy
query_linear_sql_pipe = SQLPipeline(dataset=query_static_test_dataset,model=list(query_trained_linear_models),embed_type="static")
query_linear_results, query_linear_acc = query_linear_sql_pipe.batch_predict(dataloader=query_static_test_loader)
print(f"\nAccuracy of Linear model on query split test set: {query_linear_acc:.2%}")

Generating SQL queries: 100%|██████████| 11/11 [00:01<00:00,  9.63it/s]


Accuracy of Linear model on query split test set: 0.00%


### Training Feedforward models using question and query split

In [21]:
# Question split
# Model init
question_ff_tag_model = FFTagger(input_dim=question_context_train_val_dataset.get_vector_dim(),
    tag_vocab_size=question_context_train_val_dataset.get_tag_vocab_size(),
    hidden_sizes=[512,256,128],
    dropout=0.2
    )
question_ff_classifier_model = FFClassifier(
    input_dim=question_context_train_val_dataset.get_vector_dim(),
    sql_vocab_size=question_context_train_val_dataset.get_sql_vocab_size(),
    hidden_sizes=[512,256,128],
    dropout=0.2
    )
# Training
question_trained_ff_model = train_dual_model(
    tagger_model=question_ff_tag_model,
    classifier_model=question_ff_classifier_model,
    train_loader=question_context_train_val_loader,
    epochs=50,
    warmup_epochs=15,
    gamma=0.95
)

Epoch 1/50: 100%|██████████| 152/152 [00:01<00:00, 114.22it/s]


Epoch 1/50, Training Loss: 5.3318, Learning Rate: 0.001


Epoch 2/50: 100%|██████████| 152/152 [00:01<00:00, 122.48it/s]


Epoch 2/50, Training Loss: 4.1862, Learning Rate: 0.001


Epoch 3/50: 100%|██████████| 152/152 [00:01<00:00, 128.88it/s]


...

Epoch 48/50, Training Loss: 0.1956, Learning Rate: 0.00018402591023557573


Epoch 49/50: 100%|██████████| 152/152 [00:01<00:00, 127.90it/s]


Epoch 49/50, Training Loss: 0.1904, Learning Rate: 0.00017482461472379692


Epoch 50/50: 100%|██████████| 152/152 [00:01<00:00, 126.84it/s]

Epoch 50/50, Training Loss: 0.1799, Learning Rate: 0.00016608338398760707


In [22]:
# Evaluating on the test set to get accuracy
question_ff_sql_pipe = SQLPipeline(dataset=question_context_test_dataset,model=list(question_trained_ff_model),embed_type="context")
question_ff_results, question_ff_acc = question_ff_sql_pipe.batch_predict(dataloader=question_context_test_loader)
print(f"\nAccuracy of Feedforward model on question split test set: {question_ff_acc:.2%}")

Generating SQL queries: 100%|██████████| 14/14 [00:00<00:00, 17.93it/s]


Accuracy of Feedforward model on question split test set: 19.02%


In [23]:
# Query split
# Model init
query_ff_tag_model = FFTagger(input_dim=query_context_train_val_dataset.get_vector_dim(),
    tag_vocab_size=query_context_train_val_dataset.get_tag_vocab_size(),
    hidden_sizes=[512,256,128],
    dropout=0.2
  )
query_ff_classifier_model = FFClassifier(
    input_dim=query_context_train_val_dataset.get_vector_dim(),
    sql_vocab_size=query_context_train_val_dataset.get_sql_vocab_size(),
    hidden_sizes=[512,256,128],
    dropout=0.2
    )
# Training
query_trained_ff_model = train_dual_model(
    tagger_model=query_ff_tag_model,
    classifier_model=query_ff_classifier_model,
    train_loader=query_context_train_val_loader,
    epochs=50,
    warmup_epochs=15,
    gamma=0.95
)

Epoch 1/50: 100%|██████████| 155/155 [00:01<00:00, 129.32it/s]


Epoch 1/50, Training Loss: 5.2312, Learning Rate: 0.001


Epoch 2/50: 100%|██████████| 155/155 [00:01<00:00, 93.26it/s]


Epoch 2/50, Training Loss: 4.0993, Learning Rate: 0.001


Epoch 3/50: 100%|██████████| 155/155 [00:01<00:00, 106.12it/s]


...

Epoch 48/50, Training Loss: 0.1753, Learning Rate: 0.00018402591023557573


Epoch 49/50: 100%|██████████| 155/155 [00:01<00:00, 90.42it/s]


Epoch 49/50, Training Loss: 0.1650, Learning Rate: 0.00017482461472379692


Epoch 50/50: 100%|██████████| 155/155 [00:01<00:00, 106.33it/s]

Epoch 50/50, Training Loss: 0.1535, Learning Rate: 0.00016608338398760707


In [24]:
# Evaluating on the test set to get accuracy
query_ff_sql_pipe = SQLPipeline(dataset=query_context_test_dataset,model=list(query_trained_ff_model),embed_type="context")
query_ff_results, query_ff_acc = query_ff_sql_pipe.batch_predict(dataloader=query_context_test_loader)
print(f"\nAccuracy of Feedforward model on query split test set: {query_ff_acc:.2%}")

Generating SQL queries: 100%|██████████| 11/11 [00:00<00:00, 11.79it/s]


Accuracy of Feedforward model on query split test set: 0.00%


### Training LSTM model using question and query split

In [25]:
# Question split
question_lstm_model = LSTMTaggerClassifer(
    input_dim=question_context_train_val_dataset.get_vector_dim(),
    hidden_dim=256,
    tag_vocab_size=question_context_train_val_dataset.get_tag_vocab_size(),
    sql_vocab_size=question_context_train_val_dataset.get_sql_vocab_size(),
    num_layers=3
)
# Training
question_trained_lstm_model = train_model(
    model=question_lstm_model,
    train_loader=question_context_train_val_loader,
    epochs=80,
    lr=1e-3,
    weight_decay=0.1,
    amsgrad=True,
    gamma=0.90,
    warmup_epochs=25
)


Epoch 1/80: 100%|██████████| 152/152 [00:02<00:00, 51.34it/s]


Epoch 1/80, Training Loss: 5.9574, Learning Rate: 0.001


Epoch 2/80: 100%|██████████| 152/152 [00:02<00:00, 57.83it/s]


Epoch 2/80, Training Loss: 4.4973, Learning Rate: 0.001


Epoch 3/80: 100%|██████████| 152/152 [00:02<00:00, 53.40it/s]


...

Epoch 78/80, Training Loss: 0.0262, Learning Rate: 3.7571021261363684e-06


Epoch 79/80: 100%|██████████| 152/152 [00:02<00:00, 57.17it/s]


Epoch 79/80, Training Loss: 0.0260, Learning Rate: 3.3813919135227317e-06


Epoch 80/80: 100%|██████████| 152/152 [00:02<00:00, 56.86it/s]

Epoch 80/80, Training Loss: 0.0258, Learning Rate: 3.0432527221704586e-06


In [26]:
# Evaluating on the test set to get accuracy
question_lstm_sql_pipe = SQLPipeline(dataset=question_context_test_dataset,model=question_trained_lstm_model,embed_type="context")
question_lstm_results, question_lstm_acc = question_lstm_sql_pipe.batch_predict(dataloader=question_context_test_loader)
print(f"\nAccuracy of LSTM model on question split test set: {question_lstm_acc:.2%}")

Generating SQL queries: 100%|██████████| 14/14 [00:00<00:00, 17.36it/s]


Accuracy of LSTM model on question split test set: 33.33%


In [27]:
# Query split
query_lstm_model = LSTMTaggerClassifer(
    input_dim=query_context_train_val_dataset.get_vector_dim(),
    hidden_dim=256,
    tag_vocab_size=query_context_train_val_dataset.get_tag_vocab_size(),
    sql_vocab_size=query_context_train_val_dataset.get_sql_vocab_size(),
    num_layers=3
)
# Training
query_trained_lstm_model = train_model(
    model=query_lstm_model,
    train_loader=query_context_train_val_loader,
    epochs=150,
    lr=1e-3,
    weight_decay=0.1,
    amsgrad=True,
    gamma=0.90,
    warmup_epochs=25

)

Epoch 1/150: 100%|██████████| 155/155 [00:02<00:00, 54.32it/s]


Epoch 1/150, Training Loss: 5.9136, Learning Rate: 0.001


Epoch 2/150: 100%|██████████| 155/155 [00:03<00:00, 49.08it/s]


Epoch 2/150, Training Loss: 4.3145, Learning Rate: 0.001


Epoch 3/150: 100%|██████████| 155/155 [00:02<00:00, 59.51it/s]


...

Epoch 148/150, Training Loss: 0.0250, Learning Rate: 2.3541203471209526e-09


Epoch 149/150: 100%|██████████| 155/155 [00:02<00:00, 57.74it/s]


Epoch 149/150, Training Loss: 0.0247, Learning Rate: 2.1187083124088576e-09


Epoch 150/150: 100%|██████████| 155/155 [00:02<00:00, 57.50it/s]

Epoch 150/150, Training Loss: 0.0248, Learning Rate: 1.906837481167972e-09


In [187]:
# Evaluating on the test set to get accuracy
query_lstm_sql_pipe = SQLPipeline(dataset=query_context_test_dataset,model=query_trained_lstm_model,embed_type="context")
query_lstm_results, query_lstm_acc = query_lstm_sql_pipe.batch_predict(dataloader=query_context_test_loader)
print(f"\nAccuracy of LSTM model on query split test set: {query_lstm_acc:.2%}")

Generating SQL queries: 100%|██████████| 11/11 [00:01<00:00,  8.99it/s]


Accuracy of LSTM model on query split test set: 0.00%


### Training Transformer model using question and query split

In [29]:
# Question split
question_transformer_model = TransformerTaggerClassifier(
    input_dim=question_context_train_val_dataset.get_vector_dim(),
    hidden_dim=256,
    num_heads=8,
    num_layers=2,
    tag_vocab_size=question_context_train_val_dataset.get_tag_vocab_size(),
    sql_vocab_size=question_context_train_val_dataset.get_sql_vocab_size()
)
# Training
question_trained_transformer_model = train_model(
    model=question_transformer_model,
    train_loader=question_context_train_val_loader,
    weight_decay=0.1,
    epochs=150,
    amsgrad=True,
    gamma=0.95,
    lr=1e-3
)

Epoch 1/150: 100%|██████████| 152/152 [00:02<00:00, 70.60it/s]


Epoch 1/150, Training Loss: 4.9607, Learning Rate: 0.001


Epoch 2/150: 100%|██████████| 152/152 [00:02<00:00, 64.99it/s]


Epoch 2/150, Training Loss: 3.3980, Learning Rate: 0.001


Epoch 3/150: 100%|██████████| 152/152 [00:01<00:00, 78.32it/s]


...

Epoch 148/150, Training Loss: 0.0595, Learning Rate: 8.430581474923329e-07


Epoch 149/150: 100%|██████████| 152/152 [00:01<00:00, 76.50it/s]


Epoch 149/150, Training Loss: 0.0621, Learning Rate: 8.009052401177162e-07


Epoch 150/150: 100%|██████████| 152/152 [00:01<00:00, 83.22it/s]

Epoch 150/150, Training Loss: 0.0599, Learning Rate: 7.608599781118303e-07


In [30]:
# Evaluating on the test set to get accuracy
question_transformer_sql_pipe = SQLPipeline(dataset=question_context_test_dataset,model=question_trained_transformer_model,embed_type="context")
question_transformer_results, question_transformer_acc = question_transformer_sql_pipe.batch_predict(dataloader=question_context_test_loader)
print(f"\nAccuracy of Transformer model on question split test set: {question_transformer_acc:.2%}")

Generating SQL queries:   0%|          | 0/14 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:508: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. We recommend specifying layout=torch.jagged when constructing a nested tensor, as this layout receives active development, has better operator coverage, and works with torch.compile. (Triggered internally at /pytorch/aten/src/ATen/NestedTensorImpl.cpp:178.)
  output = torch._nested_tensor_from_mask(
Generating SQL queries: 100%|██████████| 14/14 [00:00<00:00, 14.65it/s]


Accuracy of Transformer model on question split test set: 36.24%


In [31]:
# Query split
query_transformer_model = TransformerTaggerClassifier(
    input_dim=query_context_train_val_dataset.get_vector_dim(),
    hidden_dim=256,
    num_heads=8,
    num_layers=2,
    tag_vocab_size=query_context_train_val_dataset.get_tag_vocab_size(),
    sql_vocab_size=query_context_train_val_dataset.get_sql_vocab_size()
)
# Training
query_trained_transformer_model = train_model(
    model=query_transformer_model,
    train_loader=query_context_train_val_loader,
    weight_decay=0.1,
    epochs=150,
    amsgrad=True,
    gamma=0.95,
    lr=1e-3
)

Epoch 1/150: 100%|██████████| 155/155 [00:01<00:00, 82.41it/s]


Epoch 1/150, Training Loss: 4.8744, Learning Rate: 0.001


Epoch 2/150: 100%|██████████| 155/155 [00:01<00:00, 83.02it/s]


Epoch 2/150, Training Loss: 3.2214, Learning Rate: 0.001


Epoch 3/150: 100%|██████████| 155/155 [00:01<00:00, 83.69it/s]


...

Epoch 148/150, Training Loss: 0.0556, Learning Rate: 8.430581474923329e-07


Epoch 149/150: 100%|██████████| 155/155 [00:01<00:00, 82.80it/s]


Epoch 149/150, Training Loss: 0.0553, Learning Rate: 8.009052401177162e-07


Epoch 150/150: 100%|██████████| 155/155 [00:01<00:00, 81.21it/s]

Epoch 150/150, Training Loss: 0.0518, Learning Rate: 7.608599781118303e-07


In [188]:
# Evaluating on the test set to get accuracy
query_transformer_sql_pipe = SQLPipeline(dataset=query_context_test_dataset,model=query_trained_transformer_model,embed_type="context")
query_transformer_results, query_transformer_acc = query_transformer_sql_pipe.batch_predict(dataloader=query_context_test_loader)
print(f"\nAccuracy of Transformer model on query split test set: {query_lstm_acc:.2%}")

Generating SQL queries: 100%|██████████| 11/11 [00:00<00:00, 11.89it/s]


Accuracy of Transformer model on query split test set: 0.00%


# Experiment

For our experiment, we wanted look how each of our 4 tagging and classification models perform when noise is introduced in the input text in the form of typos or rearranging words in the input sentence.

**Methodology**
* To do this, we took all 4 models and trained them using the static embeddings we used for our Linear models. Linear models did not need re-training as they were already trained on the static embeddings. The reason to use to static embeddings was to keep training data consistent between all 4 models and also so that models do not have access to contextual information in the input.
* We then evaluated the all 4 models on the question split test set to record queries which each model tagged correctly and queries which each model classified correctly
* These queries were then incrementally altered slightly by introducing typos and changing order of the words
* All models were evaluated again on these modified queries and the results were compared against the unmodified version of these queries

In [162]:
# Evaluating Linear Models on the question split test set to get results
question_static_linear_sql_pipe = SQLPipeline(dataset=question_static_test_dataset,model=list(question_trained_linear_models),embed_type="static")
question_static_linear_results, question_static_linear_acc = question_static_linear_sql_pipe.batch_predict(dataloader=question_static_test_loader)

print(f"\nAccuracy of Linear model on question split test set: {question_static_linear_acc:.2%}")

Generating SQL queries: 100%|██████████| 14/14 [00:01<00:00, 13.87it/s]


Accuracy of Linear model on question split test set: 11.19%


In [109]:
# Training feedforward models using static embeddings from the question split training and validation set
question_static_ff_tag_model = FFTagger(input_dim=question_static_train_val_dataset.get_vector_dim(),
    tag_vocab_size=question_static_train_val_dataset.get_tag_vocab_size(),
    hidden_sizes=[512,256,128],
    dropout=0.2
  )
question_static_ff_classifier_model = FFClassifier(
    input_dim=question_static_train_val_dataset.get_vector_dim(),
    sql_vocab_size=question_static_train_val_dataset.get_sql_vocab_size(),
    hidden_sizes=[512,256,128],
    dropout=0.2
    )
question_static_trained_ff_model = train_dual_model(
    tagger_model=question_static_ff_tag_model,
    classifier_model=question_static_ff_classifier_model,
    train_loader=question_static_train_val_loader,
    epochs=50,
    warmup_epochs=15,
    gamma=0.95
)

Epoch 1/50: 100%|██████████| 152/152 [00:03<00:00, 39.66it/s]


Epoch 1/50, Training Loss: 5.4730, Learning Rate: 0.001


Epoch 2/50: 100%|██████████| 152/152 [00:04<00:00, 34.57it/s]


Epoch 2/50, Training Loss: 4.4822, Learning Rate: 0.001


Epoch 3/50: 100%|██████████| 152/152 [00:04<00:00, 32.82it/s]


...

Epoch 48/50, Training Loss: 0.3670, Learning Rate: 0.00018402591023557573


Epoch 49/50: 100%|██████████| 152/152 [00:04<00:00, 31.69it/s]


Epoch 49/50, Training Loss: 0.3589, Learning Rate: 0.00017482461472379692


Epoch 50/50: 100%|██████████| 152/152 [00:03<00:00, 39.69it/s]

Epoch 50/50, Training Loss: 0.3357, Learning Rate: 0.00016608338398760707


In [163]:
# Evaluating Feedforward models on the test set
question_static_ff_sql_pipe = SQLPipeline(dataset=question_static_test_dataset,model=list(question_static_trained_ff_model),embed_type="static")
question_static_ff_results, question_static_ff_acc = question_static_ff_sql_pipe.batch_predict(dataloader=question_static_test_loader)
# question_static_ff_sql_pipe = SQLPipeline(dataset=question_static_train_val_dataset,model=list(question_static_trained_ff_model),embed_type="static")
# question_static_ff_results, question_static_ff_acc = question_static_ff_sql_pipe.batch_predict(dataloader=question_static_train_val_loader)
print(f"\nAccuracy of Feedforward model on question split test set: {question_static_ff_acc:.2%}")

Generating SQL queries: 100%|██████████| 14/14 [00:01<00:00,  9.84it/s]


Accuracy of Feedforward model on question split test set: 11.63%


In [111]:
# Training LSTM model using static embeddings from the question split traing and validation set
question_static_lstm_model = LSTMTaggerClassifer(
    input_dim=question_static_train_val_dataset.get_vector_dim(),
    hidden_dim=256,
    tag_vocab_size=question_static_train_val_dataset.get_tag_vocab_size(),
    sql_vocab_size=question_static_train_val_dataset.get_sql_vocab_size(),
    num_layers=3
)
# Training
question_static_trained_lstm_model = train_model(
    model=question_static_lstm_model,
    train_loader=question_static_train_val_loader,
    epochs=50,
    lr=1e-3,
    weight_decay=0.1,
    amsgrad=True,
    gamma=0.99,
    warmup_epochs=25
)

Epoch 1/50: 100%|██████████| 152/152 [00:06<00:00, 25.30it/s]


Epoch 1/50, Training Loss: 5.8671, Learning Rate: 0.001


Epoch 2/50: 100%|██████████| 152/152 [00:06<00:00, 23.84it/s]


Epoch 2/50, Training Loss: 4.3318, Learning Rate: 0.001


Epoch 3/50: 100%|██████████| 152/152 [00:06<00:00, 24.81it/s]


...

Epoch 48/50, Training Loss: 0.0266, Learning Rate: 0.0007936142836436553


Epoch 49/50: 100%|██████████| 152/152 [00:06<00:00, 22.98it/s]


Epoch 49/50, Training Loss: 0.0271, Learning Rate: 0.0007856781408072188


Epoch 50/50: 100%|██████████| 152/152 [00:05<00:00, 26.40it/s]

Epoch 50/50, Training Loss: 0.0283, Learning Rate: 0.0007778213593991466


In [164]:
# Evaluating LSTM model on the test set
question_static_lstm_sql_pipe = SQLPipeline(dataset=question_static_test_dataset,model=question_static_trained_lstm_model,embed_type="static")
question_static_lstm_results, question_static_lstm_acc = question_static_lstm_sql_pipe.batch_predict(dataloader=question_static_test_loader)
# question_static_lstm_sql_pipe = SQLPipeline(dataset=question_static_train_val_dataset,model=question_static_trained_lstm_model,embed_type="static")
# question_static_lstm_results, question_static_lstm_acc = question_static_lstm_sql_pipe.batch_predict(dataloader=question_static_train_val_loader)
print(f"\nAccuracy of LSTM model on question split test set: {question_static_lstm_acc:.2%}")

Generating SQL queries: 100%|██████████| 14/14 [00:01<00:00,  7.03it/s]


Accuracy of LSTM model on question split test set: 33.33%


In [240]:
# Training Transformer model using static embeddings from the question split training and validation set
question_static_transformer_model = TransformerTaggerClassifier(
    input_dim=question_static_train_val_dataset.get_vector_dim(),
    hidden_dim=256,
    num_heads=8,
    num_layers=2,
    tag_vocab_size=question_static_train_val_dataset.get_tag_vocab_size(),
    sql_vocab_size=question_static_train_val_dataset.get_sql_vocab_size()
)
# Training
question_static_trained_transformer_model = train_model(
    model=question_static_transformer_model,
    train_loader=question_static_train_val_loader,
    weight_decay=0.01,
    epochs=100,
    amsgrad=True,
    gamma=0.90,
    lr=1e-3,
    warmup_epochs=50
)

Epoch 1/100: 100%|██████████| 152/152 [00:05<00:00, 25.78it/s]


Epoch 1/100, Training Loss: 5.1272, Learning Rate: 0.001


Epoch 2/100: 100%|██████████| 152/152 [00:05<00:00, 27.28it/s]


Epoch 2/100, Training Loss: 3.7137, Learning Rate: 0.001


Epoch 3/100: 100%|██████████| 152/152 [00:05<00:00, 25.89it/s]


...

Epoch 98/100, Training Loss: 0.2426, Learning Rate: 6.362685441135951e-06


Epoch 99/100: 100%|██████████| 152/152 [00:05<00:00, 26.06it/s]


Epoch 99/100, Training Loss: 0.2414, Learning Rate: 5.7264168970223554e-06


Epoch 100/100: 100%|██████████| 152/152 [00:05<00:00, 30.33it/s]

Epoch 100/100, Training Loss: 0.2507, Learning Rate: 5.15377520732012e-06


In [330]:
# Evaluating Transformer model on the question test set
question_static_transformer_sql_pipe = SQLPipeline(dataset=question_static_test_dataset,model=question_static_trained_transformer_model,embed_type="static")
question_static_transformer_results, question_static_transformer_acc = question_static_transformer_sql_pipe.batch_predict(dataloader=question_static_test_loader)
# question_static_transformer_sql_pipe = SQLPipeline(dataset=question_static_train_val_dataset,model=question_static_trained_transformer_model,embed_type="static")
# question_static_transformer_results, question_static_transformer_acc = question_static_transformer_sql_pipe.batch_predict(dataloader=question_static_train_val_loader)
print(f"\nAccuracy of Transformer model on question split test set: {question_static_transformer_acc:.2%}")

Generating SQL queries: 100%|██████████| 14/14 [00:01<00:00, 13.68it/s]


Accuracy of Transformer model on question split test set: 14.32%


In [242]:
# Getting the examples all models got right for the variable tagging task in the question split test set
correct_linear_tagging_examples = [item for item in question_linear_results if item['predicted_tags'] == item['raw_item']['tagging_labels']]
correct_ff_tagging_examples = [item for item in question_static_ff_results if item['predicted_tags'] == item['raw_item']['tagging_labels']]
correct_lstm_tagging_examples = [item for item in question_static_lstm_results if item['predicted_tags'] == item['raw_item']['tagging_labels']]
correct_transformer_tagging_examples = [item for item in question_static_transformer_results if item['predicted_tags'] == item['raw_item']['tagging_labels']]

In [243]:
# Getting the input text of all correctly tagged queries for all models
linear_correct_tagging_queries = {" ".join(example['tokens']) for example in correct_linear_tagging_examples}
ff_correct_tagging_queries = {" ".join(example['tokens']) for example in correct_ff_tagging_examples}
lstm_correct_tagging_queries = {" ".join(example['tokens']) for example in correct_lstm_tagging_examples}
transformer_correct_tagging_queries = {" ".join(example['tokens']) for example in correct_transformer_tagging_examples}
correct_queries = transformer_correct_tagging_queries & lstm_correct_tagging_queries & ff_correct_tagging_queries & linear_correct_tagging_queries

In [352]:
# Functions to introduce noise in each query by adding typos, rearraging words
import random
import re

def is_uppercase_word(word):
    return word.isupper() or re.fullmatch(r'[A-Z.]+', word)

def is_to_from(word):
  return re.fullmatch(r'to',word) or re.fullmatch(r'from',word)

def typo(word):
    if len(word) > 2:
        i = random.randint(0, len(word)-2)
        return word[:i] + word[i+1] + word[i] + word[i+2:]
    return word

def delete_char(word):
    if len(word) > 1:
        i = random.randint(0, len(word)-1)
        return word[:i] + word[i+1:]
    return word


def change_sentence(sentence, change_type):
    words = sentence.split()
    lowercase_indices = [i for i, w in enumerate(words) if not is_uppercase_word(w)]
    if not lowercase_indices:
        return sentence

    i = random.choice(lowercase_indices)
    word = words[i]

    # operation = random.choice(['typo', 'delete', 'swap'])
    # operation = random.choice(['typo','swap'])
    operation = random.choice(change_type)
    # operation = random.choice(['swap'])

    if operation == 'typo':
        words[i] = typo(word)
    elif operation == 'delete':
        words[i] = delete_char(word)
    elif operation == 'swap' and i < len(words) - 1 and not is_uppercase_word(words[i+1]):
        words[i], words[i+1] = words[i+1], words[i]

    return ' '.join(words)

def modify_sentences(sentences, change_type=['typo', 'swap', 'delete']):

  return [change_sentence(s, change_type) for s in sentences]


In [353]:
# Addimg noise to inputs by swapping the order words in the sentence and evaluating all models
from copy import deepcopy
incorrect_queries = deepcopy(correct_queries)
change_results = []
for i in range(25):
  incorrect_queries = modify_sentences(incorrect_queries, change_type=['swap'])
  # Evaluating all models on the noisy queries that all of them previously tagged correctly
  linear_correct_result = []
  linear_incorrect_result = []
  ff_correct_result = []
  ff_incorrect_result = []
  lstm_correct_result = []
  lstm_incorrect_result = []
  transformer_correct_result = []
  transformer_incorrect_result = []
  linear_equal = 0
  ff_equal = 0
  lstm_equal = 0
  transformer_equal = 0
  tag_result = []
  for correct_query, incorrect_query in zip(correct_queries, incorrect_queries):
    result_item = {"correct_query":correct_query, "incorrect_query":incorrect_query, "linear":0, "ff":0, "lstm":0, "transformer":0}
    lin_correct_predict = question_linear_sql_pipe.predict(correct_query)
    lin_incorrect_predict = question_linear_sql_pipe.predict(incorrect_query)
    if(lin_correct_predict["predicted_tags"] == lin_incorrect_predict["predicted_tags"]):
      linear_equal += 1
      linear_correct_result.append((correct_query, incorrect_query))
      result_item["linear"] = 1
    else:
      linear_incorrect_result.append((correct_query, incorrect_query))

    ff_correct_predict = question_static_ff_sql_pipe.predict(correct_query)
    ff_incorrect_predict = question_static_ff_sql_pipe.predict(incorrect_query)
    if(ff_correct_predict["predicted_tags"] == ff_incorrect_predict["predicted_tags"]):
      ff_equal += 1
      ff_correct_result.append((correct_query, incorrect_query))
      result_item["ff"] = 1
    else:

      ff_incorrect_result.append((correct_query, incorrect_query))

    lstm_correct_predict = question_static_lstm_sql_pipe.predict(correct_query)
    lstm_incorrect_predict = question_static_lstm_sql_pipe.predict(incorrect_query)
    if(lstm_correct_predict["predicted_tags"] == lstm_incorrect_predict["predicted_tags"]):
      lstm_equal += 1
      lstm_correct_result.append((correct_query, incorrect_query))
      result_item["lstm"] = 1
    else:
      lstm_incorrect_result.append((correct_query, incorrect_query))

    transformer_correct_predict = question_static_transformer_sql_pipe.predict(correct_query)
    transformer_incorrect_predict = question_static_transformer_sql_pipe.predict(incorrect_query)
    if(transformer_correct_predict["predicted_tags"] == transformer_incorrect_predict["predicted_tags"]):
      transformer_equal += 1
      transformer_correct_result.append((correct_query, incorrect_query))
      result_item["transformer"] = 1
    else:
      transformer_incorrect_result.append((correct_query, incorrect_query))
    tag_result.append(result_item)
  print(f"For {i=}:")
  print(f"Linear model tagged {linear_equal} queries correctly out of {len(correct_queries)}")
  print(f"Linear Accuracy = {linear_equal/len(correct_queries):.2%}")
  print(f"Feedforward model tagged {ff_equal} queries correctly out of {len(correct_queries)}")
  print(f"Feedforward Accuracy = {ff_equal/len(correct_queries):.2%}")
  print(f"LSTM model tagged {lstm_equal} queries correctly out of {len(correct_queries)}")
  print(f"LSTM Accuracy = {lstm_equal/len(correct_queries):.2%}")
  print(f"Transformer model tagged {transformer_equal} queries correctly out of {len(correct_queries)}")
  print(f"Transformer Accuracy = {transformer_equal/len(correct_queries):.2%}")
  print("*"*50)
  change_results.append({"changes":i+1,"linear":linear_equal/len(correct_queries)*100, "ff":ff_equal/len(correct_queries)*100, "lstm":lstm_equal/len(correct_queries)*100, "transformer":transformer_equal/len(correct_queries)*100})


For i=0:
Linear model tagged 48 queries correctly out of 48
Linear Accuracy = 100.00%
Feedforward model tagged 48 queries correctly out of 48
Feedforward Accuracy = 100.00%
LSTM model tagged 47 queries correctly out of 48
LSTM Accuracy = 97.92%
Transformer model tagged 48 queries correctly out of 48
Transformer Accuracy = 100.00%
**************************************************
For i=1:
Linear model tagged 48 queries correctly out of 48
Linear Accuracy = 100.00%
Feedforward model tagged 48 queries correctly out of 48
Feedforward Accuracy = 100.00%
LSTM model tagged 47 queries correctly out of 48
LSTM Accuracy = 97.92%
Transformer model tagged 48 queries correctly out of 48
Transformer Accuracy = 100.00%
**************************************************
For i=2:
Linear model tagged 48 queries correctly out of 48
Linear Accuracy = 100.00%
Feedforward model tagged 48 queries correctly out of 48
Feedforward Accuracy = 100.00%
LSTM model tagged 48 queries correctly out of 48
LSTM Accurac

In [354]:
import pandas as pd
order_changes_df = pd.DataFrame(change_results)
order_changes_df

,changes,linear,ff,lstm,transformer
0,1,100.0,100.0,97.916667,100.0
1,2,100.0,100.0,97.916667,100.0
2,3,100.0,100.0,100.000000,100.0
3,4,100.0,100.0,100.000000,100.0
4,5,100.0,100.0,97.916667,100.0
5,6,100.0,100.0,97.916667,100.0
6,7,100.0,100.0,97.916667,100.0
7,8,100.0,100.0,97.916667,100.0
8,9,100.0,100.0,97.916667,100.0
9,10,100.0,100.0,97.916667,100.0


In [355]:
# Adding noise to inputs by rearranging letter in words and Evaluating all models
from copy import deepcopy
incorrect_queries = deepcopy(correct_queries)
change_results = []
for i in range(25):
  incorrect_queries = modify_sentences(incorrect_queries, change_type=['typo'])
  # Evaluating all models on the noisy queries that all of them previously tagged correctly
  linear_correct_result = []
  linear_incorrect_result = []
  ff_correct_result = []
  ff_incorrect_result = []
  lstm_correct_result = []
  lstm_incorrect_result = []
  transformer_correct_result = []
  transformer_incorrect_result = []
  linear_equal = 0
  ff_equal = 0
  lstm_equal = 0
  transformer_equal = 0
  tag_result = []
  for correct_query, incorrect_query in zip(correct_queries, incorrect_queries):
    result_item = {"correct_query":correct_query, "incorrect_query":incorrect_query, "linear":0, "ff":0, "lstm":0, "transformer":0}
    lin_correct_predict = question_linear_sql_pipe.predict(correct_query)
    lin_incorrect_predict = question_linear_sql_pipe.predict(incorrect_query)
    if(lin_correct_predict["predicted_tags"] == lin_incorrect_predict["predicted_tags"]):
      linear_equal += 1
      linear_correct_result.append((correct_query, incorrect_query))
      result_item["linear"] = 1
    else:
      linear_incorrect_result.append((correct_query, incorrect_query))

    ff_correct_predict = question_static_ff_sql_pipe.predict(correct_query)
    ff_incorrect_predict = question_static_ff_sql_pipe.predict(incorrect_query)
    if(ff_correct_predict["predicted_tags"] == ff_incorrect_predict["predicted_tags"]):
      ff_equal += 1
      ff_correct_result.append((correct_query, incorrect_query))
      result_item["ff"] = 1
    else:

      ff_incorrect_result.append((correct_query, incorrect_query))

    lstm_correct_predict = question_static_lstm_sql_pipe.predict(correct_query)
    lstm_incorrect_predict = question_static_lstm_sql_pipe.predict(incorrect_query)
    if(lstm_correct_predict["predicted_tags"] == lstm_incorrect_predict["predicted_tags"]):
      lstm_equal += 1
      lstm_correct_result.append((correct_query, incorrect_query))
      result_item["lstm"] = 1
    else:
      lstm_incorrect_result.append((correct_query, incorrect_query))

    transformer_correct_predict = question_static_transformer_sql_pipe.predict(correct_query)
    transformer_incorrect_predict = question_static_transformer_sql_pipe.predict(incorrect_query)
    if(transformer_correct_predict["predicted_tags"] == transformer_incorrect_predict["predicted_tags"]):
      transformer_equal += 1
      transformer_correct_result.append((correct_query, incorrect_query))
      result_item["transformer"] = 1
    else:
      transformer_incorrect_result.append((correct_query, incorrect_query))
    tag_result.append(result_item)
  print(f"For {i=}:")
  print(f"Linear model tagged {linear_equal} queries correctly out of {len(correct_queries)}")
  print(f"Linear Accuracy = {linear_equal/len(correct_queries):.2%}")
  print(f"Feedforward model tagged {ff_equal} queries correctly out of {len(correct_queries)}")
  print(f"Feedforward Accuracy = {ff_equal/len(correct_queries):.2%}")
  print(f"LSTM model tagged {lstm_equal} queries correctly out of {len(correct_queries)}")
  print(f"LSTM Accuracy = {lstm_equal/len(correct_queries):.2%}")
  print(f"Transformer model tagged {transformer_equal} queries correctly out of {len(correct_queries)}")
  print(f"Transformer Accuracy = {transformer_equal/len(correct_queries):.2%}")
  print("*"*50)
  change_results.append({"changes":i+1,"linear":linear_equal/len(correct_queries)*100, "ff":ff_equal/len(correct_queries)*100, "lstm":lstm_equal/len(correct_queries)*100, "transformer":transformer_equal/len(correct_queries)*100})

For i=0:
Linear model tagged 21 queries correctly out of 48
Linear Accuracy = 43.75%
Feedforward model tagged 22 queries correctly out of 48
Feedforward Accuracy = 45.83%
LSTM model tagged 42 queries correctly out of 48
LSTM Accuracy = 87.50%
Transformer model tagged 36 queries correctly out of 48
Transformer Accuracy = 75.00%
**************************************************
For i=1:
Linear model tagged 11 queries correctly out of 48
Linear Accuracy = 22.92%
Feedforward model tagged 13 queries correctly out of 48
Feedforward Accuracy = 27.08%
LSTM model tagged 39 queries correctly out of 48
LSTM Accuracy = 81.25%
Transformer model tagged 32 queries correctly out of 48
Transformer Accuracy = 66.67%
**************************************************
For i=2:
Linear model tagged 6 queries correctly out of 48
Linear Accuracy = 12.50%
Feedforward model tagged 10 queries correctly out of 48
Feedforward Accuracy = 20.83%
LSTM model tagged 39 queries correctly out of 48
LSTM Accuracy = 81.25

In [356]:
import pandas as pd
typo_changes_df = pd.DataFrame(change_results)
typo_changes_df

,changes,linear,ff,lstm,transformer
0,1,43.750000,45.833333,87.500000,75.000000
1,2,22.916667,27.083333,81.250000,66.666667
2,3,12.500000,20.833333,81.250000,62.500000
3,4,10.416667,12.500000,68.750000,62.500000
4,5,2.083333,6.250000,66.666667,50.000000
5,6,2.083333,2.083333,66.666667,56.250000
6,7,2.083333,2.083333,66.666667,56.250000
7,8,0.000000,2.083333,64.583333,54.166667
8,9,2.083333,4.166667,70.833333,54.166667
9,10,2.083333,4.166667,66.666667,47.916667
